In [1]:
import pandas as pd
import re
import unicodedata
import emoji
from confusable_homoglyphs import confusables


In [3]:
df = pd.read_csv('dataset/datasetKaggle.csv')
# df.head()
# print(df.columns)

In [5]:
kolom_dihapus = ['title', 'tanggal', 'author','predicted_label'] 

# video_id: id unik untuk setiap video (id video youtube)
# channel_name: Nama channel youtube
# komentar: Original comment text
# komentar_clean: _____
# label: (1 =  promosi judol, 0 = bukan promosi judol)

df = df.drop(columns=kolom_dihapus, errors='ignore')

df.head()
print(df.columns)

# df.to_csv('dataset_kaggle_judi2.csv', index=False)


Index(['video_id', 'channel_name', 'komentar', 'label', 'komentar_clean'], dtype='object')



- Menyesuaikan kolom dengan format data yg digunakan.
- Memastikan kolom yang digunakan: komentar,label.


In [ ]:
def normalisasi_unicode(teks):
    if not isinstance(teks, str):
        teks = str(teks)

    teks = unicodedata.normalize("NFKC", teks)

    hasil = []
    for ch in teks:
        if ord(ch) < 128:
            hasil.append(ch)
            continue
        try:
            info = confusables.is_confusable(
                ch,
                greedy=False,
                preferred_aliases=['latin']
            )
        except Exception:
            info = False

        if info:
            pengganti = None
            for entry in info:
                for h in entry.get('homoglyphs', []):
                    cand = h.get('c')
                    if (
                        isinstance(cand, str)
                        and len(cand) == 1
                        and ord(cand) < 128
                    ):
                        pengganti = cand
                        break
                if pengganti:
                    break

            hasil.append(pengganti if pengganti else ch)
        else:
            hasil.append(ch)

    return ''.join(hasil)


def preprocess_komentar(teks):
    if pd.isna(teks):
        return ""
    teks = str(teks).lower()
    teks = emoji.demojize(teks, language='id')
    teks = normalisasi_unicode(teks)

    return teks



- normalisasi_unicode: menormalkan teks ke NFKC dan mencoba memetakan huruf mirip (homoglyph) ke ASCII jika tersedia.
- preprocess_komentar: lowercasing, konversi emoji ke token teks, lalu normalisasi unicode.


In [15]:
df['komentar_stage1'] = df['komentar'].apply(preprocess_komentar)
df[df['label'] == 1][['komentar', 'komentar_stage1']].sample(10)


,komentar,komentar_stage1
8197,terima kasi rezekinya 𝗦𝗚𝗜𝟴𝟴,terima kasi rezekinya SGI88
3774,gue udah nemu jalan ninja gue : alexis17,gue udah nemu jalan ninja gue : alexis17
2870,"salam 𝗦𝗚𝗜𝟴𝟴 ! gampang jepe , sehat","salam SGI88 ! gampang jepe , sehat"
6754,"gue sih percaya aja alexis17 , gak ngecewain .","gue sih percaya aja alexis17 , gak ngecewain ."
7383,"D𝑂𝑅𝐴𝟩𝟽 gua WD gede, lgnsung plg","dORA77 gua wd gede, lgnsung plg"
1513,"maen jepek , gacor banget","maen jepek , gacor banget"
2867,"gacu​r​ dikit , langsung jepe aja","gacu​r​ dikit , langsung jepe aja"
2721,anjir alexis17 situs kesukaan teman gw,anjir alexis17 situs kesukaan teman gw
675,coba dengerin orang ngegas,coba dengerin orang ngegas
7128,gila modal usaha gua bertambah berkat alexis17 .,gila modal usaha gua bertambah berkat alexis17 .


In [7]:
def hapus_combining_marks(teks):
    if not isinstance(teks, str):
        teks = str(teks)

    hasil = []
    for ch in teks:
        if unicodedata.category(ch).startswith('M'):
            continue
        hasil.append(ch)

    return ''.join(hasil)



- hapus_combining_marks: menghilangkan karakter kategori (combining marks) yang bisa mengubah tampilan huruf.
- Mengurangi variasi visual yang tidak penting bagi model


In [8]:
def hapus_tanda_baca(teks):
    if not isinstance(teks, str):
        teks = str(teks)

    tanda_baca = [
        '.', ',', '!', '?', ':', ';',
        '(', ')', '"', "'", '[', ']', '{', '}', '-','@'
    ]

    for t in tanda_baca:
        teks = teks.replace(t, ' ')

    teks = ' '.join(teks.split())
    return teks


In [9]:
def preprocess_komentar_bersih(teks):
    teks = preprocess_komentar(teks)
    teks = hapus_combining_marks(teks)
    teks = hapus_tanda_baca(teks)

    return teks


- hapus_tanda_baca: mengganti tanda baca menjadi spasi dan merapikan whitespace.


In [16]:
df['komentar_clean'] = df['komentar'].apply(preprocess_komentar_bersih)

df[['komentar', 'komentar_clean']].sample(10)


,komentar,komentar_clean
9031,​ @maizarindra498 ok coba jabarin 1 per 1,​ maizarindra498 ok coba jabarin 1 per 1
7712,WEHHHH UDH CRASH AJAAAA💔💔💔,wehhhh udh crash ajaaaa patah_hati patah_hati ...
5726,Ganteng Banget❤,ganteng banget hati_merah
5704,selesai 𝘼𝙇𝙀𝙓𝙄𝙎17,selesai ALEXIS17
2396,𝗪𝗘𝗧𝗢𝗡𝟴𝟴 uhuy !,WETON88 uhuy
3888,"Klu kerja masih di cariin ortu, mending gk usa...",klu kerja masih di cariin ortu mending gk usah...
9272,"rezeki 𝐴ея𝙊𝟪8-bikin hariku menyenangkan , than...",rezeki AeяO88 bikin hariku menyenangkan thank you
5274,"ok min , weton88",ok min weton88
1402,MAKASIIIII KO🫡,makasiiiii ko wajah_memberi_hormat
1255,gue sih udah ga heran kalo alexis17 ! trending...,gue sih udah ga heran kalo alexis17 trending s...


In [14]:
df[['komentar', 'komentar_clean']].sample(10)

# df.to_csv('dataset_preprocessed.csv', index=False)

,komentar,komentar_clean
7243,orang bilang alexis17 tuh bener-bener legit .,orang bilang alexis17 tuh bener bener legit
2661,Gas seminggu 2x,gas seminggu 2x
993,"𝘼𝙇𝙀𝙓𝙄𝙎17 emang daun banget , gila sih . sih ?",ALEXIS17 emang daun banget gila sih sih
1663,Hadir 🎉🎉❤❤,hadir party_popper party_popper hati_merah hat...
582,Komen paling akurat 🥂,komen paling akurat mendentingkan_gelas
3311,Interior terlalu biasa...sayang banget.,interior terlalu biasa sayang banget
2388,"bro , lo udah cobain alexis17 ? dijamin nagih ...",bro lo udah cobain alexis17 dijamin nagih abis
7626,kebangkitan cuan weton88,kebangkitan cuan weton88
5282,mulyono kedua,mulyono kedua
1957,"sebulan main alexis17 , gue udah liburan raja ...",sebulan main alexis17 gue udah liburan raja ampat


In [ ]:
train_df = df[['komentar_clean', 'label']].dropna()
train_df['label'] = train_df['label'].astype(int)
# train_df.to_csv('dataset/train_clean_label.csv', index=False)